In [94]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn.model_selection import StratifiedShuffleSplit


In [95]:
base_dir='drive/MyDrive/brain/Training'
categories=['glioma','meningioma','notumor','pituitary']

In [96]:
import os
data = []
labels = []

for category in categories:
    path = os.path.join(base_dir, category)
    class_num = categories.index(category)
    if not os.path.exists(path):
        print(f"Directory {path} does not exist!")
        continue
    class_num = categories.index(category)
    for img in os.listdir(path):
        try:
            img_array = load_img(os.path.join(path, img), target_size=(64, 64))
            img_array = img_to_array(img_array)
            data.append(img_array)
            labels.append(class_num)
        except Exception as e:
            print(f"Error loading image: {e}")

In [97]:
data = np.array(data, dtype='float32')
labels = np.array(labels)

In [98]:
data /= 255.0
from sklearn.model_selection import StratifiedShuffleSplit
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [99]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [100]:
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

In [101]:
pca = PCA(n_components=50)  # Number of principal components to keep
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)

In [102]:
svm = SVC(kernel='linear', C=1)
svm.fit(X_train_pca, y_train)

SVC(C=1, kernel='linear')

In [103]:
y_pred = svm.predict(X_test_pca)

In [104]:
unique_classes=np.unique(y_test)
unique_classes_names=[categories[i] for i in unique_classes]

In [105]:
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=unique_classes_names))

Classification Report:
              precision    recall  f1-score   support

      glioma       0.78      0.82      0.80       288
  meningioma       0.69      0.64      0.67       265
     notumor       0.86      0.84      0.85       291
   pituitary       0.88      0.93      0.90       299

    accuracy                           0.81      1143
   macro avg       0.80      0.81      0.80      1143
weighted avg       0.81      0.81      0.81      1143



In [106]:
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

Accuracy Score:
0.8101487314085739


In [107]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def preprocess_image(image_path, target_size=(64, 64)):
    """Load and preprocess a single image."""
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = img_array.reshape(1, -1)
    img_array = scaler.transform(img_array)
    img_array = pca.transform(img_array)
    return img_array

def predict_image(model, image_array, categories):
    """Predict the class of a single image."""
    prediction = model.predict(image_array)
    class_idx = prediction[0]
    return categories[class_idx]

# Path to the image you want to test
test_image_path = 'drive/MyDrive/brain/Training/glioma/Tr-gl_0014.jpg'

# Preprocess the image
test_image_array = preprocess_image(test_image_path)

# Predict the class of the image
predicted_class = predict_image(svm, test_image_array, categories)

print(f"The predicted class for the test image is: {predicted_class}")

The predicted class for the test image is: glioma
